In [18]:
import numpy as np
from scipy.io import arff
import pandas as pd
import time
import skmultilearn.adapt as skmad
from skmultilearn.dataset import load_dataset
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import hamming_loss,coverage_error,label_ranking_loss,label_ranking_average_precision_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from scipy.special import softmax

In [2]:
#Load dataset

X, y, feature_names, label_names = load_dataset('yeast', 'undivided')
X = X.toarray()
y = y.toarray()

classifiers = []
metrics = ['Hamming loss','Coverage','Ranking loss','Average precision']
hl=[]
ce=[]
rl=[]
ap=[]

nfolds = 10 # Numero de k-folds
kf = KFold(n_splits=nfolds)

yeast:undivided - exists, not redownloading


In [22]:
class MLkNN:

    def __init__(self,k,s=1.0,smax=False):
        self.k = k
        self.s = s
        self.smax = smax
        
    def fit(self,X_train,Y_train):
        
        N_train,N_labels = Y_train.shape
        
        # Calculo das probabilidades a priori
        prior_proba = np.zeros((2,N_labels))
        
        for l in np.arange(N_labels):
            prior_proba[1,l] = (self.s + sum(Y_train[:,l]))/(self.s*2+N_train)
            prior_proba[0,l] = 1 - prior_proba[1,l]
            
        # Calculo das probabilidades a posteriori
        
        cond_proba = np.zeros((2,self.k+1,N_labels))
        
        # Calculo das distancias
        # dist(i,j) = distancia entre a i-esima amostra de teste e a j-esima amostra de treinamento
        #dist = np.zeros((N_train,N_train))       ## Considera o elemento como vizinho a ele mesmo
        dist = np.ones((N_train,N_train))*np.Inf  ## Desconsidera o  elemento como vizinho a ele mesmo

        for i in np.arange(N_train):
            for j in np.arange(i+1,N_train):
                # Distancia euclidiana
                dist[i,j] = np.sqrt(np.sum((X_train[i] - X_train[j])**2))
                dist[j,i] = dist[i,j]

        ind_neigh = np.argsort(dist,axis=1)[:,:self.k] # indices dos k vizinhos em ordem crescente de distancia
        
        for l in np.arange(N_labels):
            
            c = np.zeros((2,self.k+1))
            
            for i in np.arange(N_train):
                delta = sum(Y_train[ind_neigh[i],l])
                if(Y_train[i,l] == 1):
                    c[1,delta] = c[1,delta] + 1
                else:
                    c[0,delta] = c[0,delta] + 1
            
            for j in np.arange(self.k+1):
                cond_proba[1,j,l] = (self.s + c[1,j])/(self.s*(self.k+1) + sum(c[1,:]))
                cond_proba[0,j,l] = (self.s + c[0,j])/(self.s*(self.k+1) + sum(c[0,:]))
            print cond_proba[:,j,l]
                
        self.X_train = X_train
        self.Y_train = Y_train
        self.N_labels = N_labels
        self.prior_proba = prior_proba
        self.cond_proba = cond_proba
            
    def predict(self,X_test):
        
        X_train = self.X_train
        Y_train = self.Y_train
        N_labels = self.N_labels
        N_train = X_train.shape[0]
        N_test = X_test.shape[0]
        
        Y_test = np.zeros((N_test,N_labels))
        post_proba = np.zeros((N_test,N_labels))
        
        prior_proba = self.prior_proba
        cond_proba = self.cond_proba
        
         # Calculo das distancias
        # dist(i,j) = distancia entre a i-esima amostra de teste e a j-esima amostra de treinamento
        dist = np.zeros((N_test,N_train))

        for i in np.arange(N_test):
            for j in np.arange(N_train):
                # Distancia euclidiana
                dist[i,j] = np.sqrt(np.sum((X_test[i] - X_train[j])**2))

        ind_neigh = np.argsort(dist,axis=1)[:,:self.k] # indices dos k vizinhos em ordem crescente de distancia
        
        for t in np.arange(N_test):
            
            for l in np.arange(N_labels):
                c = sum(Y_train[ind_neigh[t],l])
                proba_1 = prior_proba[1,l]*cond_proba[1,c,l]
                proba_0 = prior_proba[0,l]*cond_proba[0,c,l]
                if(proba_1 > proba_0):
                    Y_test[t,l] = 1
                post_proba[t,l] = proba_1/(proba_1+proba_0)
        
        return Y_test,post_proba

In [24]:
#ML-kNN implementado

k = 12
test_hl = np.zeros(nfolds)
test_ce = np.zeros(nfolds)
test_rl = np.zeros(nfolds)
test_ap = np.zeros(nfolds)

i=0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    classifier = MLkNN(k)

    # train
    classifier.fit(X_train, y_train)

    # predict
    pred,proba = classifier.predict(X_test)
    
    test_hl[i] = hamming_loss(y_test,pred)
    test_ce[i] = coverage_error(y_test,proba)
    test_rl[i] = label_ranking_loss(y_test,proba)
    test_ap[i] = label_ranking_average_precision_score(y_test,proba)
    i = i+1
    
classifiers.append('ML-kNN impl.')
hl.append(np.mean(test_hl))
ce.append(np.mean(test_ce))
rl.append(np.mean(test_rl))
ap.append(np.mean(test_ap))

[0.00066313 0.03751804]
[0.00079808 0.01160338]
[0.00232019 0.02643172]
[0.00642398 0.1975    ]
[0.00324675 0.23903177]
[0.00060277 0.04428044]
[0.00055218 0.0025641 ]
[0.00056561 0.00230947]
[0.00049213 0.00591716]
[0.00050968 0.0041841 ]
[0.00051867 0.003663  ]
[0.04166667 0.1594906 ]
[0.04056437 0.15605875]
[0.00046425 0.0212766 ]
[0.00133601 0.03551136]
[0.00079936 0.01578947]
[0.00153257 0.0234375 ]
[0.00421941 0.19127086]
[0.00450741 0.2345679 ]
[0.00060606 0.04900181]
[0.00055432 0.00755668]
[0.0005685  0.00452489]
[0.0004914 0.0060241]
[0.00051125 0.00408163]
[0.00052165 0.00352113]
[0.05063291 0.1559466 ]
[0.04920914 0.15134804]
[0.00046318 0.02380952]


KeyboardInterrupt: 

In [23]:
#ML-kNN com softmax

k = 12
test_hl = np.zeros(nfolds)
test_ce = np.zeros(nfolds)
test_rl = np.zeros(nfolds)
test_ap = np.zeros(nfolds)

i=0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    classifier2 = MLkNN(k,smax=True)

    # train
    classifier2.fit(X_train, y_train)

    # predict
    pred,proba = classifier2.predict(X_test)
    
    test_hl[i] = hamming_loss(y_test,pred)
    test_ce[i] = coverage_error(y_test,proba)
    test_rl[i] = label_ranking_loss(y_test,proba)
    test_ap[i] = label_ranking_average_precision_score(y_test,proba)
    i = i+1
    
classifiers.append('ML-kNN c/ softmax')
hl.append(np.mean(test_hl))
ce.append(np.mean(test_ce))
rl.append(np.mean(test_rl))
ap.append(np.mean(test_ap))

[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.14285714]
[0.07142857 0.14285714]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.14285714]
[0.07142857 0.14285714]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.14285714]
[0.07142857 0.14285714]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07142857]
[0.07142857 0.07

KeyboardInterrupt: 

In [ ]:
#ML-kNN scikit

k = 12
test_hl = np.zeros(nfolds)
test_ce = np.zeros(nfolds)
test_rl = np.zeros(nfolds)
test_ap = np.zeros(nfolds)

i=0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    classifier3 = skmad.MLkNN(k=12)

    # train
    classifier3.fit(X_train, y_train)

    # predict
    pred = classifier3.predict(X_test)
    proba = classifier3.predict_proba(X_test).toarray()

    test_hl[i] = hamming_loss(y_test,pred)
    test_ce[i] = coverage_error(y_test,proba)
    test_rl[i] = label_ranking_loss(y_test,proba)
    test_ap[i] = label_ranking_average_precision_score(y_test,proba)
    i = i+1
    
classifiers.append('ML-kNN scikit')
hl.append(np.mean(test_hl))
ce.append(np.mean(test_ce))
rl.append(np.mean(test_rl))
ap.append(np.mean(test_ap))

In [ ]:
#Binary Relevance

k = 12
test_hl = np.zeros(nfolds)
test_ce = np.zeros(nfolds)
test_rl = np.zeros(nfolds)
test_ap = np.zeros(nfolds)

i=0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    classifier4 = BinaryRelevance(classifier = KNeighborsClassifier(n_neighbors=k))

    # train
    classifier4.fit(X_train, y_train)

    # predict
    pred = classifier4.predict(X_test)
    proba = classifier4.predict_proba(X_test).toarray()
    
    test_hl[i] = hamming_loss(y_test,pred)
    test_ce[i] = coverage_error(y_test,proba)
    test_rl[i] = label_ranking_loss(y_test,proba)
    test_ap[i] = label_ranking_average_precision_score(y_test,proba)
    i = i+1
    
classifiers.append('Binary Relevance')
hl.append(np.mean(test_hl))
ce.append(np.mean(test_ce))
rl.append(np.mean(test_rl))
ap.append(np.mean(test_ap))

In [ ]:
# Mostrar os resultados

results = [hl,ce,rl,ap]
table = dict([(metrics[i],results[i]) for i in range(4)])
  
# Creates pandas DataFrame. 
df = pd.DataFrame(table, columns=metrics,index=classifiers) 
  
# print the data 
df 